**18.Para cada lenguaje indicar cuántos usuarios lo comprenden, cuantos lo manejan a nivel lectura y escritura base, cuantos hacen de él, un uso avanzado. (Para resolver deberá mapear los niveles de babel a esas categorías propuestas y darles un nombre). (⭐)**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 336 kB in 2s (176 kB/s)
Reading package lists... Done
Building dependency tree       
Reading s

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_languages = sqlContext.read.csv('/content/drive/MyDrive/datosTP1/languages.csv', header=True, inferSchema=True)
rdd_languages = df_languages.rdd

In [ ]:
rdd_languages.take(5)

[Row(babel_user=4502458, babel_lang='FR', babel_level='2'),
 Row(babel_user=5928200, babel_lang='aa', babel_level='0'),
 Row(babel_user=46918, babel_lang='ab', babel_level='0'),
 Row(babel_user=2050449, babel_lang='ab', babel_level='1'),
 Row(babel_user=4715583, babel_lang='ace', babel_level='0')]

Categorizo que niveles tienen los hablantes de los idiomas. Siendo B = básico, M = medio y A = avanzado. 

In [ ]:
def categorizar(nivel_lenguaje):
  if nivel_lenguaje == "1":
    return "B"
  elif nivel_lenguaje == "2" or nivel_lenguaje == "3":
    return "M"
  return "A"

lenguajes_con_categorias = rdd_languages.filter(lambda x: x[2] != "0").map( lambda x: ( (x[1].lower(), categorizar(x[2])), 1) )

In [ ]:
lenguajes_con_categorias.reduceByKey(lambda x,y: x+y).collect()

[(('fr', 'M'), 1525),
 (('ab', 'B'), 1),
 (('acf', 'B'), 1),
 (('af', 'B'), 6),
 (('af', 'M'), 3),
 (('af', 'A'), 1),
 (('agr', 'B'), 1),
 (('akk', 'B'), 1),
 (('aln', 'A'), 2),
 (('am', 'M'), 2),
 (('an', 'B'), 38),
 (('an', 'M'), 15),
 (('an', 'A'), 4),
 (('ang', 'B'), 3),
 (('ang', 'M'), 3),
 (('ar', 'B'), 92),
 (('ar', 'M'), 43),
 (('ar', 'A'), 26),
 (('arc', 'B'), 1),
 (('arc', 'A'), 2),
 (('arn', 'B'), 10),
 (('arn', 'M'), 4),
 (('arn', 'A'), 1),
 (('ary', 'A'), 1),
 (('ase', 'B'), 1),
 (('ase', 'M'), 1),
 (('ast', 'B'), 58),
 (('ast', 'M'), 48),
 (('ast', 'A'), 46),
 (('avk', 'M'), 1),
 (('ay', 'B'), 4),
 (('az', 'B'), 1),
 (('az', 'M'), 2),
 (('az', 'A'), 3),
 (('azb', 'A'), 1),
 (('bar', 'B'), 2),
 (('bar', 'M'), 2),
 (('bar', 'A'), 2),
 (('bcl', 'B'), 1),
 (('be', 'B'), 8),
 (('be', 'M'), 11),
 (('be', 'A'), 5),
 (('bew', 'A'), 1),
 (('bg', 'B'), 15),
 (('bg', 'M'), 7),
 (('bg', 'A'), 8),
 (('bjn', 'B'), 1),
 (('bn', 'M'), 1),
 (('br', 'M'), 1),
 (('br', 'A'), 1),
 (('bs', 'B